In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils import data

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from aif360.algorithms.postprocessing import (CalibratedEqOddsPostprocessing,
                                              EqOddsPostprocessing,
                                              RejectOptionClassification)
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
random_state = 1
import math

mps


pip install 'aif360[AdversarialDebiasing]'


In [3]:
# read in data and split
X = torch.load('inputs/rfw_senet50_face_embeddings.pt').cpu()
y = torch.load('inputs/rfw_senet50_labels.pt').cpu()
df = pd.read_csv('inputs/rfw_senet50_df.csv')
df['reference_ethnicity'] = df['reference_ethnicity'].str.lower()
ethnicity = df['reference_ethnicity']
df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
0,m.0c7mh2,m.0c7mh2,african,African,1.0
1,m.0c7mh2,m.0c7mh2,african,African,1.0
2,m.026tq86,m.026tq86,african,African,1.0
3,m.026tq86,m.026tq86,african,African,1.0
4,m.02wz3nc,m.02wz3nc,african,African,1.0
...,...,...,...,...,...
29311,m.0402tg,m.01npnk3,caucasian,Caucasian,0.0
29312,m.05pbbnj,m.02rrb2n,caucasian,Caucasian,0.0
29313,m.09j6df,m.07kcsqd,african,African,0.0
29314,m.0fhrbz,m.025zgjt,african,African,0.0


In [4]:
### split data into 3 sets
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)

train_X = X[train_split]
X_val = X[val_split]
test_X = X[test_split]

train_y = y[train_split]
y_val = y[val_split]
test_y = y[test_split]

train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]

ethnicity = df['reference_ethnicity'].values
ethnicity[ethnicity=='caucasian'] = 0
ethnicity[ethnicity=='african'] = 1
ethnicity = ethnicity.astype(int)

train_ethnicity = ethnicity[train_split]


ethnicity_val = ethnicity[val_split]

test_ethnicity = ethnicity[test_split]

In [5]:
### get breakdown of training set

train_matches = train_df[train_df.labels==1]
train_non_matches = train_df[train_df.labels==0]
print('non matches')
print(train_non_matches['reference_ethnicity'].value_counts())
print('matches')
print(train_matches['reference_ethnicity'].value_counts())

african_non_matches = train_non_matches[train_non_matches['reference_ethnicity']=='african']
caucasian_non_matches = train_non_matches[train_non_matches['reference_ethnicity']=='caucasian']

african_matches = train_matches[train_matches['reference_ethnicity']=='african']
caucasian_matches = train_matches[train_matches['reference_ethnicity']=='caucasian']

non matches
african      4492
caucasian    4376
Name: reference_ethnicity, dtype: int64
matches
african      4396
caucasian    4325
Name: reference_ethnicity, dtype: int64


In [6]:
np.random.seed(random_state)
african_matches_sub_idx = african_matches.index[np.random.choice(len(african_matches.index), size=4325, replace=False)]
caucasian_non_matches_sub_idx = caucasian_non_matches.index[np.random.choice(len(caucasian_non_matches.index), size=4325, replace=False)]
african_non_matches_sub_idx = african_non_matches.index[np.random.choice(len(african_non_matches.index), size=4325, replace=False)]
caucasian_matches_sub_idx = caucasian_matches.index

X_train = torch.cat([X[african_matches_sub_idx],X[caucasian_matches_sub_idx],X[african_non_matches_sub_idx],X[caucasian_non_matches_sub_idx]])
y_train = torch.cat([y[african_matches_sub_idx],y[caucasian_matches_sub_idx],y[african_non_matches_sub_idx],y[caucasian_non_matches_sub_idx]])
ethnicity_train = np.concatenate([ethnicity[african_matches_sub_idx],ethnicity[caucasian_matches_sub_idx],ethnicity[african_non_matches_sub_idx],ethnicity[caucasian_non_matches_sub_idx]])
df_train = pd.concat([df.iloc[african_matches_sub_idx],df.iloc[caucasian_matches_sub_idx],df.iloc[african_non_matches_sub_idx],df.iloc[caucasian_non_matches_sub_idx]])
df_train

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
6419,m.03j6s0,m.03j6s0,1,African,1.0
5541,m.01qd5bm,m.01qd5bm,1,African,1.0
5881,m.02qdp4v,m.02qdp4v,1,African,1.0
229,m.0272001,m.0272001,1,African,1.0
5264,m.05f962,m.05f962,1,African,1.0
...,...,...,...,...,...
18810,m.0g3f1p,m.01kxfvq,0,Caucasian,0.0
20550,m.07gb0p,m.0bgwxt,0,Caucasian,0.0
24018,m.07pyvy,m.01y7m1,0,Caucasian,0.0
20879,m.02qg85,m.070p26,0,Caucasian,0.0


In [7]:
test_split, holdout_split = train_test_split(np.arange(5864),test_size=0.1, random_state=random_state)
X_test = test_X[test_split]
X_holdout = test_X[holdout_split]
y_test = test_y[test_split]
y_holdout = test_y[holdout_split]
ethnicity_test = test_ethnicity[test_split]
ethnicity_holdout = test_ethnicity[holdout_split]
df_test = test_df.iloc[test_split]
df_holdout = test_df.iloc[holdout_split]
df_holdout

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
26599,m.08dyjb,m.01dysq,african,African,0.0
13208,m.05nqz8,m.05nqz8,caucasian,Caucasian,1.0
7459,m.0chlpf,m.0chlpf,caucasian,Caucasian,1.0
4747,m.0gzkxh,m.0gzkxh,african,African,1.0
22098,m.01tl0vh,m.027g5hs,african,African,0.0
...,...,...,...,...,...
27224,m.0glqs_8,m.05972y,african,African,0.0
5728,m.06w9s22,m.06w9s22,african,African,1.0
18206,m.04zvsy6,m.06tvw7,caucasian,Caucasian,0.0
25909,m.0h_dpyh,m.04d_dnm,african,African,0.0


In [10]:
class TrainData(data.Dataset):
    
    def __init__(self, X_data, y_data,ethnicity):
        self.X_data = X_data
        self.y_data = y_data
        self.ethnicity = ethnicity    
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index], self.ethnicity[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train),
                       ethnicity_train)
test_data = TrainData(torch.FloatTensor(X_test),torch.FloatTensor(y_test),ethnicity_test)
holdout_data = TrainData(torch.FloatTensor(X_holdout),torch.FloatTensor(y_holdout),ethnicity_holdout)
val_data = TrainData(torch.FloatTensor(X_val), 
                       torch.FloatTensor(y_val),
                       ethnicity_val)

In [11]:
### define a bunch of stuff

class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 4096
        self.layer_1 = nn.Linear(4096, 1024) 
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(1024)
        self.layer_2 = nn.Linear(1024, 512)
        self.batchnorm2 = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(p=0.1)
        self.layer_out = nn.Linear(512, 1) 

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
def val_model(model, loader, criterion):
    """Validate model on loader with criterion function"""
    y_true, y_pred, y_prot = [], [], []
    model.eval()
    with torch.no_grad():
        for inputs, labels,protected in loader:
            inputs = inputs.to(device)
            y_true.append(labels)
            y_prot.append(protected)
            y_pred.append(torch.sigmoid(model(inputs)).cpu())
    y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
    return criterion(y_true, y_pred, y_prot)
def compute_bias(y_pred, y_true, prot, metric):
    """Compute bias on the dataset"""
    def zero_if_nan(data):
        """Zero if there is a nan"""
        return 0. if torch.isnan(data) else data

    gtpr_prot = zero_if_nan(y_pred[prot * y_true == 1].mean())
    gfpr_prot = zero_if_nan(y_pred[prot * (1-y_true) == 1].mean())
    mean_prot = zero_if_nan(y_pred[prot == 1].mean())

    gtpr_unprot = zero_if_nan(y_pred[(1-prot) * y_true == 1].mean())
    gfpr_unprot = zero_if_nan(y_pred[(1-prot) * (1-y_true) == 1].mean())
    mean_unprot = zero_if_nan(y_pred[(1-prot) == 1].mean())

    if metric == "spd":
        return mean_prot - mean_unprot
    elif metric == "aod":
        return 0.5 * ((gfpr_prot - gfpr_unprot) + (gtpr_prot - gtpr_unprot))
    elif metric == "eod":
        return gtpr_prot - gtpr_unprot
    elif metric=='false_diff':
        return (np.abs(gfpr_prot - gfpr_unprot) + np.abs(gtpr_unprot - gtpr_prot))

def compute_objective(performance, bias, epsilon=0.001, margin=0.0005):
    if abs(bias) <= (epsilon-margin):
        return performance
    else:
        return 0.0
def get_best_objective(y_true, y_pred, y_prot):
    """Find the threshold for the best objective"""
    num_samples = 5
    threshs = torch.linspace(0, 1, 501)
    best_obj, best_thresh = -math.inf, 0.
    for thresh in threshs:
        indices = np.random.choice(np.arange(y_pred.size()[0]), num_samples*y_pred.size()[0], replace=True).reshape(num_samples, y_pred.size()[0])
        objs = []
        for index in indices:
            y_pred_tmp = y_pred[index]
            y_true_tmp = y_true[index]
            y_prot_tmp = y_prot[index]
            perf = (torch.mean((y_pred_tmp > thresh)[y_true_tmp.type(torch.bool)].type(torch.float32)) + torch.mean((y_pred_tmp <= thresh)[~y_true_tmp.type(torch.bool)].type(torch.float32))) / 2
            bias = compute_bias((y_pred_tmp > thresh).float().cpu(), y_true_tmp.float().cpu(), y_prot_tmp.float().cpu(), 'false_diff')
            objs.append(compute_objective(perf, bias))
        obj = float(torch.tensor(objs).mean())
        if obj > best_obj:
            best_obj, best_thresh = obj, thresh

    return best_obj, best_thresh
def AOE(tn_1,fp_1,fn_1,tp_1,tn_0,fp_0,fn_0,tp_0):
    tpr_1 = tp_1/(tp_1+fn_1)
    tpr_0 = tp_0/(tp_0+fn_0)

    fpr_1 = fp_1/(fp_1+tn_1)
    fpr_0 = fp_0/(fp_0+tn_0)

    return (np.abs(fpr_1-fpr_0) + np.abs(tpr_1 - tpr_0))/2
def to_dataframe(y_true, y_pred, y_prot):
        y_true, y_pred, y_prot = y_true.float().cpu().numpy(), y_pred.float().cpu().numpy(), y_prot.float().cpu().numpy()
        df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'y_prot': y_prot})
        dataset = StandardDataset(df, 'y_true', [1.], ['y_prot'], [[1.]])
        dataset.scores = y_pred.reshape(-1, 1)
        return dataset


In [12]:
### load pre trained model
EPOCHS = 20
BATCH_SIZE = 64
LEARNING_RATE = 1e-5
eps = 1e-5
model = torch.load('weights/rfw_senet50_logistic_regression_face_matching_0.0_betaTEST.pt')
# model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_loader = data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
val_loader = data.DataLoader(dataset=val_data, batch_size=BATCH_SIZE)
holdout_loader = data.DataLoader(dataset=holdout_data, batch_size=BATCH_SIZE)

In [13]:
# compute bias on val set
y_true, y_pred, y_prot = [], [], []
# model.eval()
with torch.no_grad():
    for inputs, labels,protected in val_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(model(inputs)[:, 0]).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)

compute_bias((y_pred>0.5).float().cpu(), y_true, y_prot, 'false_diff')


tensor(0.0189)

In [14]:
# apply model to test set and get bias before mitigating
test_true, test_pred, test_prot = [], [], []
# model.eval()
with torch.no_grad():
    for inputs, labels,protected in test_loader:
        inputs = inputs.to(device)
        test_true.append(labels)
        test_prot.append(protected)
        test_pred.append(torch.sigmoid(model(inputs)[:, 0]).cpu())
test_true, test_pred, test_prot = torch.cat(test_true), torch.cat(test_pred), torch.cat(test_prot)

print(compute_bias((test_pred>0.5).float().cpu(), test_true, test_prot, 'false_diff'))
print(compute_bias((test_pred>0.5).float().cpu(), test_true, test_prot, 'aod'))
aif_data = to_dataframe(test_true, test_pred, test_prot)

tensor(0.0340)
tensor(-0.0170)


In [15]:
# same thing as before pretty much
test_bias_df = pd.DataFrame(columns=['label','prediction','rounded','protected'])
test_bias_df['label']=test_true.cpu().numpy()
test_bias_df['prediction']=test_pred.cpu().numpy()
test_bias_df['rounded']=(test_pred>0.5).float().cpu()
test_bias_df['protected']=test_prot
unpriveleged_df = test_bias_df[test_bias_df['protected']==1]
priveleged_df = test_bias_df[test_bias_df['protected']==0]

unpriveleged_tn, unpriveleged_fp, unpriveleged_fn, unpriveleged_tp, unpriveleged_acc = confusion_mat(unpriveleged_df['rounded'], unpriveleged_df['label'])
priveleged_tn, priveleged_fp, priveleged_fn, priveleged_tp, priveleged_acc = confusion_mat(priveleged_df['rounded'], priveleged_df['label'])
unpriveleged_fnr = unpriveleged_fn/(unpriveleged_fn+unpriveleged_tp)
unpriveleged_fpr = unpriveleged_fp/(unpriveleged_fp+unpriveleged_tn)
priveleged_fnr = priveleged_fn/(priveleged_fn+priveleged_tp)
priveleged_fpr = priveleged_fp/(priveleged_fp+priveleged_tn)

print('Overall Accuracy:',round(torch.sum((test_pred>0.5).float().cpu() == test_true).item()/test_true.size()[0],3))
print('Missclassificaition Rate Ratio:',round((1-unpriveleged_acc)/(1- priveleged_acc),3))
print('FNR Ratio:',round(unpriveleged_fnr/priveleged_fnr,3),'FPR Ratio:',round(unpriveleged_fpr/priveleged_fpr,3))
print('AOE:',round(AOE(unpriveleged_tn,unpriveleged_fp,unpriveleged_fn,unpriveleged_tp,priveleged_tn,priveleged_fp,priveleged_fn,priveleged_tp),3))

Overall Accuracy: 0.938
Missclassificaition Rate Ratio: 1.156
FNR Ratio: 2.346 FPR Ratio: 0.909
AOE: 0.017


In [16]:
### apply eo post
eo = EqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                  unprivileged_groups=[{'y_prot': 1.}],seed=random_state)

eo=eo.fit(aif_data,aif_data)

eo_pred = eo.predict(aif_data).labels.reshape(-1)
eo_label = aif_data.labels.reshape(-1)
eo_prot = aif_data.protected_attributes.reshape(-1)

eo_unpriv_ids = (eo_prot==1)
eo_unpriv_pred = eo_pred[eo_unpriv_ids]
eo_unpriv_label = eo_label[eo_unpriv_ids]

eo_priv_ids = (eo_prot==0)
eo_priv_pred = eo_pred[eo_priv_ids]
eo_priv_label = eo_label[eo_priv_ids]

eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp, eo_unpriv_acc = confusion_mat(eo_unpriv_pred, eo_unpriv_label)
eo_unpriv_fpr = eo_unpriv_fp/(eo_unpriv_fp+eo_unpriv_tn)
eo_unpriv_fnr = eo_unpriv_fn/(eo_unpriv_fn+eo_unpriv_tp)

eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp, eo_priv_acc = confusion_mat(eo_priv_pred, eo_priv_label)
eo_priv_fpr = eo_priv_fp/(eo_priv_fp+eo_priv_tn)
eo_priv_fnr = eo_priv_fn/(eo_priv_fn+eo_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-eo_unpriv_acc)/(1- eo_priv_acc),3))
print('FPR Ratio:',eo_unpriv_fpr/eo_priv_fpr)
print('FNR Ratio:',eo_unpriv_fnr/eo_priv_fnr)
print('Overall Accuracy:', np.sum(eo_pred == eo_label)/len(eo_pred))
print('AOD:',compute_bias(torch.tensor(eo_pred), torch.tensor(eo_label), torch.tensor(eo_prot), 'aod'))
print('False Diff:',compute_bias(torch.tensor(eo_pred), torch.tensor(eo_label), torch.tensor(eo_prot), 'false_diff'))


Missclassificaition Rate Ratio: nan
FPR Ratio: nan
FNR Ratio: nan
Overall Accuracy: 1.0
AOD: tensor(0., dtype=torch.float64)
False Diff: tensor(0., dtype=torch.float64)


In [17]:
# apply model to holdout set and get bias before mitigating
holdout_true, holdout_pred, holdout_prot = [], [], []
# model.eval()
with torch.no_grad():
    for inputs, labels,protected in holdout_loader:
        inputs = inputs.to(device)
        holdout_true.append(labels)
        holdout_prot.append(protected)
        holdout_pred.append(torch.sigmoid(model(inputs)[:, 0]).cpu())
holdout_true, holdout_pred, holdout_prot = torch.cat(holdout_true), torch.cat(holdout_pred), torch.cat(holdout_prot)
print(compute_bias((holdout_pred>0.5).float().cpu(), holdout_true, holdout_prot, 'false_diff'))
print(compute_bias((holdout_pred>0.5).float().cpu(), holdout_true, holdout_prot, 'aod'))

tensor(0.1172)
tensor(-0.0586)


In [18]:
### apply eo prediction to hold out
holdout_aif_data = to_dataframe(holdout_true, holdout_pred, holdout_prot)

holdout_eo_pred = eo.predict(holdout_aif_data).labels.reshape(-1)
holdout_eo_label = holdout_aif_data.labels.reshape(-1)
holdout_eo_prot = holdout_aif_data.protected_attributes.reshape(-1)

holdout_eo_unpriv_ids = (holdout_eo_prot==1)
holdout_eo_unpriv_pred = holdout_eo_pred[holdout_eo_unpriv_ids]
holdout_eo_unpriv_label = holdout_eo_label[holdout_eo_unpriv_ids]

holdout_eo_priv_ids = (holdout_eo_prot==0)
holdout_eo_priv_pred = holdout_eo_pred[holdout_eo_priv_ids]
holdout_eo_priv_label = holdout_eo_label[holdout_eo_priv_ids]

holdout_eo_unpriv_tn, holdout_eo_unpriv_fp, holdout_eo_unpriv_fn, holdout_eo_unpriv_tp, holdout_eo_unpriv_acc = confusion_mat(holdout_eo_unpriv_pred, holdout_eo_unpriv_label)
holdout_eo_unpriv_fpr = holdout_eo_unpriv_fp/(holdout_eo_unpriv_fp+holdout_eo_unpriv_tn)
holdout_eo_unpriv_fnr = holdout_eo_unpriv_fn/(holdout_eo_unpriv_fn+holdout_eo_unpriv_tp)

holdout_eo_priv_tn, holdout_eo_priv_fp, holdout_eo_priv_fn, holdout_eo_priv_tp, holdout_eo_priv_acc = confusion_mat(holdout_eo_priv_pred, holdout_eo_priv_label)
holdout_eo_priv_fpr = holdout_eo_priv_fp/(holdout_eo_priv_fp+holdout_eo_priv_tn)
holdout_eo_priv_fnr = holdout_eo_priv_fn/(holdout_eo_priv_fn+holdout_eo_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-holdout_eo_unpriv_acc)/(1- holdout_eo_priv_acc),3))
print('FPR Ratio:',holdout_eo_unpriv_fpr/holdout_eo_priv_fpr)
print('FNR Ratio:',holdout_eo_unpriv_fnr/holdout_eo_priv_fnr)
print('Overall Accuracy:', np.sum(holdout_eo_pred == holdout_eo_label)/len(holdout_eo_pred))
print('AOD:',compute_bias(torch.tensor(holdout_eo_pred), torch.tensor(holdout_eo_label), torch.tensor(holdout_eo_prot), 'aod'))
print('False Diff:',compute_bias(torch.tensor(holdout_eo_pred), torch.tensor(holdout_eo_label), torch.tensor(holdout_eo_prot), 'false_diff'))


Missclassificaition Rate Ratio: nan
FPR Ratio: nan
FNR Ratio: nan
Overall Accuracy: 1.0
AOD: tensor(0., dtype=torch.float64)
False Diff: tensor(0., dtype=torch.float64)


In [19]:
### apply cpp post
cost_constraint = 'weighted'
cpp = CalibratedEqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                             unprivileged_groups=[{'y_prot': 1.}],
                                             cost_constraint=cost_constraint,seed=random_state)
cpp=cpp.fit(aif_data,aif_data)
cpp_pred = cpp.predict(aif_data).labels.reshape(-1)
cpp_label = aif_data.labels.reshape(-1)
cpp_prot = aif_data.protected_attributes.reshape(-1)

cpp_unpriv_ids = (cpp_prot==1)
cpp_unpriv_pred = cpp_pred[cpp_unpriv_ids]
cpp_unpriv_label = cpp_label[cpp_unpriv_ids]

cpp_priv_ids = (cpp_prot==0)
cpp_priv_pred = cpp_pred[cpp_priv_ids]
cpp_priv_label = cpp_label[cpp_priv_ids]

cpp_unpriv_tn, cpp_unpriv_fp, cpp_unpriv_fn, cpp_unpriv_tp, cpp_unpriv_acc = confusion_mat(cpp_unpriv_pred, cpp_unpriv_label)
cpp_unpriv_fpr = cpp_unpriv_fp/(cpp_unpriv_fp+cpp_unpriv_tn)
cpp_unpriv_fnr = cpp_unpriv_fn/(cpp_unpriv_fn+cpp_unpriv_tp)

cpp_priv_tn, cpp_priv_fp, cpp_priv_fn, cpp_priv_tp, cpp_priv_acc = confusion_mat(cpp_priv_pred, cpp_priv_label)
cpp_priv_fpr = cpp_priv_fp/(cpp_priv_fp+cpp_priv_tn)
cpp_priv_fnr = cpp_priv_fn/(cpp_priv_fn+cpp_priv_tp)


print('Missclassificaition Rate Ratio:',round((1-cpp_unpriv_acc)/(1- cpp_priv_acc),3))
print('FPR Ratio:',cpp_unpriv_fpr/cpp_priv_fpr)
print('FNR Ratio:',cpp_unpriv_fnr/cpp_priv_fnr)
print('Overall Accuracy:', np.sum(cpp_pred == cpp_label)/len(eo_pred))
print('AOD:',compute_bias(torch.tensor(cpp_pred), torch.tensor(cpp_label), torch.tensor(cpp_prot), 'aod'))
print('False Diff:',compute_bias(torch.tensor(cpp_pred), torch.tensor(cpp_label), torch.tensor(cpp_prot), 'false_diff'))

Missclassificaition Rate Ratio: 1.002
FPR Ratio: 0.7671016823354775
FNR Ratio: 2.3458723302135827
Overall Accuracy: 0.934242941064999
AOD: tensor(-0.0259, dtype=torch.float64)
False Diff: tensor(0.0519, dtype=torch.float64)


In [20]:
### cpp holdout predict
holdout_cpp_pred = cpp.predict(holdout_aif_data).labels.reshape(-1)
holdout_cpp_label = holdout_aif_data.labels.reshape(-1)
holdout_cpp_prot = holdout_aif_data.protected_attributes.reshape(-1)

holdout_cpp_unpriv_ids = (holdout_cpp_prot==1)
holdout_cpp_unpriv_pred = holdout_cpp_pred[holdout_cpp_unpriv_ids]
holdout_cpp_unpriv_label = holdout_cpp_label[holdout_cpp_unpriv_ids]

holdout_cpp_priv_ids = (holdout_cpp_prot==0)
holdout_cpp_priv_pred = holdout_cpp_pred[holdout_cpp_priv_ids]
holdout_cpp_priv_label = holdout_cpp_label[holdout_cpp_priv_ids]

holdout_cpp_unpriv_tn, holdout_cpp_unpriv_fp, holdout_cpp_unpriv_fn, holdout_cpp_unpriv_tp, holdout_cpp_unpriv_acc = confusion_mat(holdout_cpp_unpriv_pred, holdout_cpp_unpriv_label)
holdout_cpp_unpriv_fpr = holdout_cpp_unpriv_fp/(holdout_cpp_unpriv_fp+holdout_cpp_unpriv_tn)
holdout_cpp_unpriv_fnr = holdout_cpp_unpriv_fn/(holdout_cpp_unpriv_fn+holdout_cpp_unpriv_tp)

holdout_cpp_priv_tn, holdout_cpp_priv_fp, holdout_cpp_priv_fn, holdout_cpp_priv_tp, holdout_cpp_priv_acc = confusion_mat(holdout_cpp_priv_pred, holdout_cpp_priv_label)
holdout_cpp_priv_fpr = holdout_cpp_priv_fp/(holdout_cpp_priv_fp+holdout_cpp_priv_tn)
holdout_cpp_priv_fnr = holdout_cpp_priv_fn/(holdout_cpp_priv_fn+holdout_cpp_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-holdout_cpp_unpriv_acc)/(1- holdout_cpp_priv_acc),3))
print('FPR Ratio:',holdout_cpp_unpriv_fpr/holdout_cpp_priv_fpr)
print('FNR Ratio:',holdout_cpp_unpriv_fnr/holdout_cpp_priv_fnr)
print('Overall Accuracy:', np.sum(holdout_cpp_pred == holdout_cpp_label)/len(holdout_cpp_pred))
print('AOD:',compute_bias(torch.tensor(holdout_cpp_pred), torch.tensor(holdout_cpp_label), torch.tensor(holdout_cpp_prot), 'aod'))
print('False Diff:',compute_bias(torch.tensor(holdout_cpp_pred), torch.tensor(holdout_cpp_label), torch.tensor(holdout_cpp_prot), 'false_diff'))


Missclassificaition Rate Ratio: 0.823
FPR Ratio: 0.4507042253521127
FNR Ratio: 4.548192771084337
Overall Accuracy: 0.9284497444633731
AOD: tensor(-0.0664, dtype=torch.float64)
False Diff: tensor(0.1328, dtype=torch.float64)


In [21]:
### apply roc post
roc = RejectOptionClassification(unprivileged_groups=[{'y_prot': 1.}],
                                         privileged_groups=[{'y_prot': 0.}],
                                         low_class_thresh=0.01, high_class_thresh=0.99,
                                         num_class_thresh=100, num_ROC_margin=50,
                                         metric_name="Average odds difference",
                                         metric_ub=0.05, metric_lb=-0.05)

roc = roc.fit(aif_data, aif_data)

roc_pred = roc.predict(aif_data).labels.reshape(-1)
roc_label = aif_data.labels.reshape(-1)
roc_prot = aif_data.protected_attributes.reshape(-1)

roc_unpriv_ids = (roc_prot==1)
roc_unpriv_pred = roc_pred[roc_unpriv_ids]
roc_unpriv_label = roc_label[roc_unpriv_ids]

roc_priv_ids = (roc_prot==0)
roc_priv_pred = roc_pred[roc_priv_ids]
roc_priv_label = roc_label[roc_priv_ids]

roc_unpriv_tn, roc_unpriv_fp, roc_unpriv_fn, roc_unpriv_tp, roc_unpriv_acc = confusion_mat(roc_unpriv_pred, roc_unpriv_label)
roc_unpriv_fpr = roc_unpriv_fp/(roc_unpriv_fp+roc_unpriv_tn)
roc_unpriv_fnr = roc_unpriv_fn/(roc_unpriv_fn+roc_unpriv_tp)

roc_priv_tn, roc_priv_fp, roc_priv_fn, roc_priv_tp, roc_priv_acc = confusion_mat(roc_priv_pred, roc_priv_label)
roc_priv_fpr = roc_priv_fp/(roc_priv_fp+roc_priv_tn)
roc_priv_fnr = roc_priv_fn/(roc_priv_fn+roc_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-roc_unpriv_acc)/(1- roc_priv_acc),3))
print('FPR Ratio:',roc_unpriv_fpr/roc_priv_fpr)
print('FNR Ratio:',roc_unpriv_fnr/roc_priv_fnr)
print('Overall Accuracy:', np.sum(roc_pred == roc_label)/len(eo_pred))
print('AOD:',compute_bias(torch.tensor(roc_pred), torch.tensor(roc_label), torch.tensor(roc_prot), 'aod'))
print('False Diff:',compute_bias(torch.tensor(roc_pred), torch.tensor(roc_label), torch.tensor(roc_prot), 'false_diff'))

Missclassificaition Rate Ratio: 1.253
FPR Ratio: 2.203555976203353
FNR Ratio: 0.7406518589623942
Overall Accuracy: 0.944097024824711
AOD: tensor(0.0292, dtype=torch.float64)
False Diff: tensor(0.0585, dtype=torch.float64)


In [22]:
### roc predict on holdout
holdout_roc_pred = roc.predict(holdout_aif_data).labels.reshape(-1)
holdout_roc_label = holdout_aif_data.labels.reshape(-1)
holdout_roc_prot = holdout_aif_data.protected_attributes.reshape(-1)

holdout_roc_unpriv_ids = (holdout_roc_prot==1)
holdout_roc_unpriv_pred = holdout_roc_pred[holdout_roc_unpriv_ids]
holdout_roc_unpriv_label = holdout_roc_label[holdout_roc_unpriv_ids]

holdout_roc_priv_ids = (holdout_roc_prot==0)
holdout_roc_priv_pred = holdout_roc_pred[holdout_roc_priv_ids]
holdout_roc_priv_label = holdout_roc_label[holdout_roc_priv_ids]

holdout_roc_unpriv_tn, holdout_roc_unpriv_fp, holdout_roc_unpriv_fn, holdout_roc_unpriv_tp, holdout_roc_unpriv_acc = confusion_mat(holdout_roc_unpriv_pred, holdout_roc_unpriv_label)
holdout_roc_unpriv_fpr = holdout_roc_unpriv_fp/(holdout_roc_unpriv_fp+holdout_roc_unpriv_tn)
holdout_roc_unpriv_fnr = holdout_roc_unpriv_fn/(holdout_roc_unpriv_fn+holdout_roc_unpriv_tp)

holdout_roc_priv_tn, holdout_roc_priv_fp, holdout_roc_priv_fn, holdout_roc_priv_tp, holdout_roc_priv_acc = confusion_mat(holdout_roc_priv_pred, holdout_roc_priv_label)
holdout_roc_priv_fpr = holdout_roc_priv_fp/(holdout_roc_priv_fp+holdout_roc_priv_tn)
holdout_roc_priv_fnr = holdout_roc_priv_fn/(holdout_roc_priv_fn+holdout_roc_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-holdout_roc_unpriv_acc)/(1- holdout_roc_priv_acc),3))
print('FPR Ratio:',holdout_roc_unpriv_fpr/holdout_roc_priv_fpr)
print('FNR Ratio:',holdout_roc_unpriv_fnr/holdout_roc_priv_fnr)
print('Overall Accuracy:', np.sum(holdout_roc_pred == holdout_roc_label)/len(holdout_roc_pred))
print('AOD:',compute_bias(torch.tensor(holdout_roc_pred), torch.tensor(holdout_roc_label), torch.tensor(holdout_roc_prot), 'aod'))
print('False Diff:',compute_bias(torch.tensor(holdout_roc_pred), torch.tensor(holdout_roc_label), torch.tensor(holdout_roc_prot), 'false_diff'))


Missclassificaition Rate Ratio: 0.996
FPR Ratio: 1.1267605633802817
FNR Ratio: 0.9096385542168673
Overall Accuracy: 0.9284497444633731
AOD: tensor(0.0076, dtype=torch.float64)
False Diff: tensor(0.0151, dtype=torch.float64)
